#  Test con PaliGemma su CM1-COVER

In questa sezione utilizziamo **PaliGemma** (modello multimodale Google) per estrarre i campi `Name`, `Vorname`, `Geb-Dat` dalle stesse 10 immagini già testate con GPT e DONUT.


In [ ]:
!pip install transformers accelerate pillow


In [ ]:
from google.colab import files
import os

uploaded = files.upload()  # seleziona le 10 immagini
os.makedirs("cm1_images", exist_ok=True)

for fname in uploaded.keys():
    os.rename(fname, f"cm1_images/{fname}")

print("Immagini salvate in cm1_images/")


Saving 78867180.jpg to 78867180.jpg
✅ Immagini salvate in cm1_images/


In [ ]:
id_to_file = {
    "78867961": "78867962.jpg",
    "78867179": "78867180.jpg",
    "78867223": "78867224.jpg",
    "78867406": "78867407.jpg",
    "78867679": "78867680.jpg",
    "78912579": "78912580.jpg",
    "78950724": "78950725.jpg",
    "78938299": "78938300.jpg",
    "78982956": "78982957.jpg",
    "78917020": "78917021.jpg"
}


In [ ]:
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch

model_id = "google/paligemma-3b-mix-224"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, device_map="auto"
)

device = model.device
print("Modello caricato su:", device)


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

✅ Modello caricato su: cuda:0


In [ ]:
def predict_paligemma(img_path, question="Estrarre in JSON i campi: Name, Vorname, Geb-Dat"):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, text=question, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    result = processor.decode(outputs[0], skip_special_tokens=True)
    return result


In [ ]:
import json
import re

predictions = {}

def parse_output(raw_output):
    # Regex semplice per Name / Vorname / Geb-Dat
    name = re.search(r"(?:Name)[: ]+([A-Za-zÀ-ÿ\-]+)", raw_output)
    vorname = re.search(r"(?:Vorname)[: ]+([A-Za-zÀ-ÿ\-]+)", raw_output)
    gebdat = re.search(r"(?:Geb[- ]?Dat)[: ]+([0-9\-]+)", raw_output)

    return {
        "Name": name.group(1) if name else "",
        "Vorname": vorname.group(1) if vorname else "",
        "Geb-Dat": gebdat.group(1) if gebdat else ""
    }

for doc_id, filename in id_to_file.items():
    img_path = f"cm1_images/{filename}"
    raw_output = predict_paligemma(img_path)
    parsed = parse_output(raw_output)
    predictions[doc_id] = [parsed]
    print(f"ID {doc_id} → {parsed}")


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


📄 ID 78867961 → {'Name': '', 'Vorname': '', 'Geb-Dat': ''}


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


📄 ID 78867179 → {'Name': '', 'Vorname': '', 'Geb-Dat': ''}


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


...

📄 ID 78950724 → {'Name': 'Paul', 'Vorname': 'Wilhelm', 'Geb-Dat': '1912'}


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


📄 ID 78938299 → {'Name': '', 'Vorname': '', 'Geb-Dat': ''}


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


📄 ID 78982956 → {'Name': '', 'Vorname': '', 'Geb-Dat': ''}
📄 ID 78917020 → {'Name': 'Paul', 'Vorname': '', 'Geb-Dat': ''}


In [ ]:
with open("predictions_paligemma.json", "w", encoding="utf-8") as f:
    json.dump(predictions, f, ensure_ascii=False, indent=2)

print("File salvato: predictions_paligemma.json")


✅ File salvato: predictions_paligemma.json
